In [1]:
from time import time
import sys; sys.path.append('..')
import dinr.modeling
import torch
from hydra.experimental import compose, initialize
from omegaconf import OmegaConf
from configs import trainer_conf
from collections import OrderedDict

initialize(config_path="../configs", job_name="test_app")

import numpy as np
import torch
import torch.nn as nn
import torch.autograd.profiler as profiler

from dinr.modeling.modules.inr_generator import INRGenerator
from dinr.modeling.modules.noise import mixing_noise

device = 'cuda'

In [5]:
cfg = compose(config_name="config", overrides=['model=stylegan2_inrgan', 'system=stylegan2_noppl_nomix_inr'])
kwargs = dict(cfg.model.generator)
kwargs.pop('_target_')
# kwargs = {**kwargs, **{'no_fourier_embs': True}}
kwargs = {**kwargs, **{'interpolation_type': 'nearest'}}
kwargs = {**kwargs, **{
    'use_noise_injection': False
}}
# kwargs = {**kwargs, **{'interpolation_type': 'bilinear'}}
kwargs_sg2 = {**kwargs, **{'fallback': True, 'is_multiscale': False}}

batch_size = 64
# batch_size = 150
num_runs = 20

times = {
    'sg2': [],
    #'ms_conv': [],
    'inr_gan': []
}

model_name = 'inr_gan'
# model_name = 'sg2'

with torch.no_grad():
    z = mixing_noise(batch_size, 512, 0.0, device)
    
    if model_name == 'inr_gan':
        gen = INRGenerator(**kwargs).to(device)
    else:
        gen = INRGenerator(**kwargs_sg2).to(device)
    
    with profiler.profile(record_shapes=True, use_cuda=True) as prof:
        for i in range(num_runs):
            torch.cuda.synchronize()
            start = time()
            
            with profiler.record_function("forward"):
                fake_img = gen(z, return_latents=False)[0].contiguous()
            torch.cuda.synchronize()
            
            if i > 3:
                times[model_name].append(time() - start)

np.mean(times[model_name])

0.23658183217048645

In [6]:
len(times[model_name]) * batch_size / np.sum(times[model_name])

270.51950444732415

In [4]:
len(times[model_name]) * batch_size / np.sum(times[model_name])

268.7019915817746

In [8]:
# f"{np.mean(times['stylegan2']) * 1000:0.2f} vs {np.mean(times['ms_conv']) * 1000:0.2f} vs {np.mean(times['ms_fmm']) * 1000:0.2f}"
f"{np.mean(times['stylegan2']) * 1000:0.2f} vs {np.mean(times['ms_conv']) * 1000:0.2f}"

'593.91 vs 391.13'

In [3]:
# f"{np.mean(times['stylegan2']) * 1000:0.2f} vs {np.mean(times['ms_conv']) * 1000:0.2f} vs {np.mean(times['ms_fmm']) * 1000:0.2f}"
f"{np.mean(times['stylegan2']) * 1000:0.2f} vs {np.mean(times['ms_conv']) * 1000:0.2f}"

'694.40 vs 500.05'